<a href="https://colab.research.google.com/github/Zayed-Rahat/ML_Lab_university/blob/main/Apriori_Algorithm_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

In [ ]:

df = pd.read_csv("aprior_test.csv", low_memory=False)

In [ ]:
df.head()

,milk,bread,biscuit,cornflakes,bournvita,jam,maggi,tea,coffee,cock,sugar
0,t,t,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,t,t,t,t,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,t,NaN,NaN,t,NaN,NaN,t,NaN,NaN,NaN
3,t,t,NaN,NaN,NaN,t,t,NaN,NaN,NaN,NaN
4,NaN,NaN,t,NaN,NaN,NaN,t,t,NaN,NaN,NaN


In [ ]:
item_list = list(df.columns)
item_dict = dict()

for i, item in enumerate(item_list):
    item_dict[item] = i + 1

item_dict

{'milk': 1,
 'bread': 2,
 'biscuit': 3,
 'cornflakes': 4,
 'bournvita': 5,
 'jam': 6,
 'maggi': 7,
 'tea': 8,
 'coffee': 9,
 'cock': 10,
 'sugar': 11}

**Extracting the transactions from the data.**

In [ ]:
transactions = []

for i, row in df.iterrows():
    transaction = set()

    for item in item_dict:
        if row[item] == 't':
            transaction.add(item_dict[item])
    transactions.append(transaction)

transactions

[{1, 2, 3},
 {1, 2, 3, 4},
 {2, 5, 8},
 {1, 2, 6, 7},
 {3, 7, 8},
 {2, 5, 8},
 {4, 7, 8},
 {2, 3, 7, 8},
 {2, 6, 7, 8},
 {1, 2},
 {3, 4, 9, 10},
 {3, 4, 9, 10},
 {5, 9, 11},
 {2, 9, 10},
 {2, 3, 11},
 {4, 9, 11},
 {2, 5, 11},
 {2, 9, 11},
 {2, 9, 11},
 {1, 4, 8, 9}]


Utility Functions

In [ ]:
def get_support(transactions, item_set):
    match_count = 0
    for transaction in transactions:
        if item_set.issubset(transaction):
            match_count += 1

    return float(match_count/len(transactions))

self_join performs join based on the last level valid sets. It joins each sets together by performing union and if the length exceeds the current level, it will skip that set.

In [ ]:
def self_join(frequent_item_sets_per_level, level):
    current_level_candidates = list()
    last_level_items = frequent_item_sets_per_level[level - 1]

    if len(last_level_items) == 0:
        return current_level_candidates

    for i in range(len(last_level_items)):
        for j in range(i+1, len(last_level_items)):
            itemset_i = last_level_items[i][0]
            itemset_j = last_level_items[j][0]
            union_set = itemset_i.union(itemset_j)

            if union_set not in current_level_candidates and len(union_set) == level:
                current_level_candidates.append(union_set)

    return current_level_candidates


pruning function prunes the candidate sets evaluated after completing the self-join part. For each itemset, it finds all its subsets by dropping a single elements from it and checks if that subset was present in the previous level or not. If that subset was not present in the previous level, then the current set is not valid and must not be used, and is thus pruned.

In [ ]:
def get_single_drop_subsets(item_set):
    single_drop_subsets = list()
    for item in item_set:
        temp = item_set.copy()
        temp.remove(item)
        single_drop_subsets.append(temp)

    return single_drop_subsets

def is_valid_set(item_set, prev_level_sets):
    single_drop_subsets = get_single_drop_subsets(item_set)

    for single_drop_set in single_drop_subsets:
        if single_drop_set not in prev_level_sets:
            return False
    return True

def pruning(frequent_item_sets_per_level, level, candidate_set):
    post_pruning_set = list()
    if len(candidate_set) == 0:
        return post_pruning_set

    prev_level_sets = list()
    for item_set, _ in frequent_item_sets_per_level[level - 1]:
        prev_level_sets.append(item_set)

    for item_set in candidate_set:
        if is_valid_set(item_set, prev_level_sets):
            post_pruning_set.append(item_set)

    return post_pruning_set


Apriori Algorithm

In [ ]:
from collections import defaultdict

def apriori(min_support):
    frequent_item_sets_per_level = defaultdict(list)
    print("level : 1", end = " ")

    for item in range(1, len(item_list) + 1):
        support = get_support(transactions, {item})
        if support >= min_support:
            frequent_item_sets_per_level[1].append(({item}, support))

    for level in range(2, len(item_list) + 1):
        print(level, end = " ")
        current_level_candidates = self_join(frequent_item_sets_per_level, level)

        post_pruning_candidates = pruning(frequent_item_sets_per_level, level, current_level_candidates)
        if len(post_pruning_candidates) == 0:
            break

        for item_set in post_pruning_candidates:
            support = get_support(transactions, item_set)
            if support >= min_support:
                frequent_item_sets_per_level[level].append((item_set, support))

    return frequent_item_sets_per_level

In [ ]:
min_support = 0.005
frequent_item_sets_per_level = apriori(min_support)

level : 1 2 3 4 5 

In [ ]:
for level in frequent_item_sets_per_level:
    print(len(frequent_item_sets_per_level[level]))

11
36
30
6


In [ ]:
for level in frequent_item_sets_per_level:
    print(frequent_item_sets_per_level[level])

[({1}, 0.25), ({2}, 0.65), ({3}, 0.35), ({4}, 0.3), ({5}, 0.2), ({6}, 0.1), ({7}, 0.25), ({8}, 0.35), ({9}, 0.4), ({10}, 0.15), ({11}, 0.3)]
[({1, 2}, 0.2), ({1, 3}, 0.1), ({1, 4}, 0.1), ({1, 6}, 0.05), ({1, 7}, 0.05), ({8, 1}, 0.05), ({1, 9}, 0.05), ({2, 3}, 0.2), ({2, 4}, 0.05), ({2, 5}, 0.15), ({2, 6}, 0.1), ({2, 7}, 0.15), ({8, 2}, 0.2), ({9, 2}, 0.15), ({2, 10}, 0.05), ({2, 11}, 0.2), ({3, 4}, 0.15), ({3, 7}, 0.1), ({8, 3}, 0.1), ({9, 3}, 0.1), ({10, 3}, 0.1), ({11, 3}, 0.05), ({4, 7}, 0.05), ({8, 4}, 0.1), ({9, 4}, 0.2), ({10, 4}, 0.1), ({11, 4}, 0.05), ({8, 5}, 0.1), ({9, 5}, 0.05), ({11, 5}, 0.1), ({6, 7}, 0.1), ({8, 6}, 0.05), ({8, 7}, 0.2), ({8, 9}, 0.05), ({9, 10}, 0.15), ({9, 11}, 0.2)]
[({1, 2, 3}, 0.1), ({1, 2, 4}, 0.05), ({1, 2, 6}, 0.05), ({1, 2, 7}, 0.05), ({1, 3, 4}, 0.05), ({8, 1, 4}, 0.05), ({1, 4, 9}, 0.05), ({1, 6, 7}, 0.05), ({8, 1, 9}, 0.05), ({2, 3, 4}, 0.05), ({2, 3, 7}, 0.05), ({8, 2, 3}, 0.05), ({11, 2, 3}, 0.05), ({8, 2, 5}, 0.1), ({2, 11, 5}, 0.05), ({2, 6

**Support Count**

In [ ]:
item_support_dict = dict()
item_list = list()

key_list = list(item_dict.keys())
val_list = list(item_dict.values())

for level in frequent_item_sets_per_level:
    for set_support_pair in frequent_item_sets_per_level[level]:
        for i in set_support_pair[0]:
            item_list.append(key_list[val_list.index(i)])
        item_support_dict[frozenset(item_list)] = set_support_pair[1]
        item_list = list()

In [ ]:
item_support_dict

{frozenset({'milk'}): 0.25,
 frozenset({'bread'}): 0.65,
 frozenset({'biscuit'}): 0.35,
 frozenset({'cornflakes'}): 0.3,
 frozenset({'bournvita'}): 0.2,
 frozenset({'jam'}): 0.1,
 frozenset({'maggi'}): 0.25,
 frozenset({'tea'}): 0.35,
 frozenset({'coffee'}): 0.4,
 frozenset({'cock'}): 0.15,
 frozenset({'sugar'}): 0.3,
 frozenset({'bread', 'milk'}): 0.2,
 frozenset({'biscuit', 'milk'}): 0.1,
 frozenset({'cornflakes', 'milk'}): 0.1,
 frozenset({'jam', 'milk'}): 0.05,
 frozenset({'maggi', 'milk'}): 0.05,
 frozenset({'milk', 'tea'}): 0.05,
 frozenset({'coffee', 'milk'}): 0.05,
 frozenset({'biscuit', 'bread'}): 0.2,
 frozenset({'bread', 'cornflakes'}): 0.05,
 frozenset({'bournvita', 'bread'}): 0.15,
 frozenset({'bread', 'jam'}): 0.1,
 frozenset({'bread', 'maggi'}): 0.15,
 frozenset({'bread', 'tea'}): 0.2,
 frozenset({'bread', 'coffee'}): 0.15,
 frozenset({'bread', 'cock'}): 0.05,
 frozenset({'bread', 'sugar'}): 0.2,
 frozenset({'biscuit', 'cornflakes'}): 0.15,
 frozenset({'biscuit', 'maggi'

In [ ]:
def find_subset(item, item_length):
    combs = []
    for i in range(1, item_length + 1):
        combs.append(list(combinations(item, i)))

    subsets = []
    for comb in combs:
        for elt in comb:
            subsets.append(elt)

    return subsets

**Association Rules**

In [ ]:
def association_rules(min_confidence, support_dict):
    rules = list()
    for item, support in support_dict.items():
        item_length = len(item)

        if item_length > 1:
            subsets = find_subset(item, item_length)

            for A in subsets:
                B = item.difference(A)

                if B:
                    A = frozenset(A)

                    AB = A | B

                    confidence = support_dict[AB] / support_dict[A]
                    if confidence >= min_confidence:
                        rules.append((A, B, confidence))

    return rules

In [ ]:
association_rules = association_rules(min_confidence = 0.6, support_dict = item_support_dict)

In [ ]:
print("Number of rules: ", len(association_rules), "\n")

for rule in association_rules:
    print('{0} -> {1} < confidence : {2} >'.format(set(rule[0]), set(rule[1]), rule[2]))

Number of rules:  86 

{'milk'} -> {'bread'} < confidence : 0.8 >
{'bournvita'} -> {'bread'} < confidence : 0.7499999999999999 >
{'jam'} -> {'bread'} < confidence : 1.0 >
{'maggi'} -> {'bread'} < confidence : 0.6 >
{'sugar'} -> {'bread'} < confidence : 0.6666666666666667 >
{'cock'} -> {'biscuit'} < confidence : 0.6666666666666667 >
{'cornflakes'} -> {'coffee'} < confidence : 0.6666666666666667 >
{'cock'} -> {'cornflakes'} < confidence : 0.6666666666666667 >
{'jam'} -> {'maggi'} < confidence : 1.0 >
{'maggi'} -> {'tea'} < confidence : 0.8 >
{'cock'} -> {'coffee'} < confidence : 1.0 >
{'sugar'} -> {'coffee'} < confidence : 0.6666666666666667 >
{'milk', 'biscuit'} -> {'bread'} < confidence : 1.0 >
{'cornflakes', 'bread'} -> {'milk'} < confidence : 1.0 >
{'jam', 'milk'} -> {'bread'} < confidence : 1.0 >
{'milk', 'maggi'} -> {'bread'} < confidence : 1.0 >
{'tea', 'milk'} -> {'cornflakes'} < confidence : 1.0 >
{'coffee', 'milk'} -> {'cornflakes'} < confidence : 1.0 >
{'jam', 'milk'} -> {'mag